In [1]:
from itertools import product

import os
import numpy as np
import pandas as pd
import networkx as nx

# Script

In [2]:
# =============================================================================
#  --------------------- INPUT PARAMETER AND PATH CLASSES ---------------------
# =============================================================================

class InputParameters():
    RUN   = 0  #sys.argv[1]
    RANGE = 10
    
    ALPHA = 0.05
    MIN_GO = 5
    MAX_GO = 500
    MIN_LVL = 0
    MAX_LVL = np.inf
    CORRECTION = 'BY'
    
    def __init__(self, network_name, feature, metric, method, aspect):
        self.network_name = network_name
        self.feature = feature
        self.metric  = metric
        self.method  = method
        self.aspect  = aspect
            
class Paths():
    DATA_DIRECTORY = "/Users/markusyoussef/Desktop/git/supplements/data"
    RAW_DATA_DIRECTORY = f"{DATA_DIRECTORY}/raw_data"
    YEAST_DIRECTORY = f"{DATA_DIRECTORY}/processed_data/yeast"
    NETWORK_DIRECTORY = f"{YEAST_DIRECTORY}/networks"
    ANNOTATION_DIRECTORY = f"{YEAST_DIRECTORY}/annotations"
    
    def __init__(self, in_parms):
        self.NETWORK_FILE    = f"{self.NETWORK_DIRECTORY}/{in_parms.network_name}.txt"
        self.ANNOTATION_FILE = f"{self.ANNOTATION_DIRECTORY}/GO_{in_parms.aspect}_systematic_SGD.csv"
        
        network_to_method = f"{in_parms.network_name}/{in_parms.feature}/{in_parms.metric}/{in_parms.method}"
        self.CLUSTER_DIRECTORY    = f"{self.YEAST_DIRECTORY}/clusterings/"   \
                                    f"{network_to_method}"
        self.PVALUE_DIRECTORY     = f"{self.YEAST_DIRECTORY}/pvalues/"       \
                                    f"{network_to_method}/{in_parms.aspect}"
        self.ENRICHMENT_DIRECTORY = f"{self.YEAST_DIRECTORY}/enrichments/"   \
                                    f"{network_to_method}/{in_parms.aspect}/{in_parms.CORRECTION}"
    
        if not os.path.exists(self.ENRICHMENT_DIRECTORY):
            os.makedirs(self.ENRICHMENT_DIRECTORY)

In [3]:
# =============================================================================
#  -------------------------------- FUNCTIONS --------------------------------
# =============================================================================

def filter_GOterms(annotation_df):
    def lvl_filter(min_lvl = InputParameters.MIN_LVL, 
                   max_lvl = InputParameters.MAX_LVL):
        return annotation_df.Level.between(min_lvl, max_lvl)
    def GO_filter(geneset,
                  min_GO = InputParameters.MIN_GO, 
                  max_GO = InputParameters.MAX_GO): 
        return min_GO <= len(geneset) <= max_GO
    
    annotation_df = annotation_df[annotation_df.Systematic_ID.isin(network_nx)]
    annotation_df = annotation_df[lvl_filter()]
    annotation_df = annotation_df.groupby('GO_ID').filter(GO_filter)
    
    return annotation_df


def nb_genes_enriched_in_cluster(cluster_enrichment, cluster, gene2GOset):
        enriched_GOterms = set(cluster_enrichment[cluster_enrichment].index)
        
        def annotated_in_enriched_term(gene):
            return not enriched_GOterms.isdisjoint(gene2GOset[gene])
        
        return sum(map(annotated_in_enriched_term, cluster))

    
def get_qvalue_threshold(pvalues, cluster_list, gene2GOset):
    correction = InputParameters.CORRECTION
    alpha      = InputParameters.ALPHA
    # union of gene2GOset across all genes in a cluster
    def cluster_to_GOset(cluster):
        if len(cluster) == 0:
            return list()
        else:
            return set.union(*map(gene2GOset.get, cluster))

    relevant_pvalues = (list(pvalues[idx_cluster][cluster_to_GOset(cluster)])
                             for idx_cluster,cluster in enumerate(cluster_list))

    sorted_pvalues = sorted(p for pvalues_s in relevant_pvalues for p in pvalues_s)
    
    m = len(sorted_pvalues) # number of tests
    
    # (pvalue == nan) <==> (cluster not annotated in GOterm)
    assert (~pvalues.isna()).sum().sum() == m, f"{sum(~pvalues.isna())} {m}"
    
    if   correction == 'Bonferroni': return min(alpha/m, alpha)
    elif correction == 'BH': c = 1
    elif correction == 'BY': c = np.log(m) + np.euler_gamma + 1/(2*m)
    else: raise Exception("Correction not known!")
        
    for k,P_k in enumerate(sorted_pvalues, 1):
        if P_k > k/(m*c) * alpha:
            # one index shift for starting numeration with 1 
            # and another one for because of overshoot in the loop 
            return min(sorted_pvalues[k-2], alpha)

In [4]:
# =============================================================================
#  ----------------------------------- INIT -----------------------------------
# =============================================================================

# Global parameters
RUN   = 0
RANGE = 10

# Input parameters
network_name = 'GI_Constanzo2016'
feature = 'GCV-G'
metric  = 'braycurtis'
method  = 'kmedoid'
aspect  = 'BP'

in_parms = InputParameters(network_name, feature, metric, method, aspect)

In [5]:
# =============================================================================
#  ---------------------------------- MAIN -----------------------------------
# =============================================================================

network_nx = nx.read_edgelist(Paths(in_parms).NETWORK_FILE)
annotation_df = pd.read_csv(Paths(in_parms).ANNOTATION_FILE)
annotation_df = filter_GOterms(annotation_df)

annotated_genes = set(annotation_df.Systematic_ID)
filtered_GOset  = set(annotation_df.GO_ID)

gene2GOset = {gene : set(go_ids.GO_ID)        for gene, go_ids in annotation_df.groupby('Systematic_ID')}
GO2geneset = {go_id: set(genes.Systematic_ID) for go_id, genes in annotation_df.groupby('GO_ID')}

GO2geneset_s = pd.Series(GO2geneset).sort_values()

cluster_coverages = np.zeros(2*RANGE+1)
GO_coverages      = np.zeros(2*RANGE+1)
gene_coverages    = np.zeros(2*RANGE+1)
    
# ----------------------- this is where the fun starts ------------------------
N = len(network_nx)
M = int(np.sqrt(N/2))

for count, n_clusters in enumerate(range(M-RANGE, M+RANGE+1)):
    with open(f"{Paths(in_parms).CLUSTER_DIRECTORY}/{RUN}_{n_clusters}.txt", 'r') as f:
                 cluster_list = [set(line.split()) for line in f]

# Keep only annotated genes in cluster
    annotated_cluster_list = [geneset & annotated_genes for geneset in cluster_list]

    pvalues = pd.read_csv(f"{Paths(in_parms).PVALUE_DIRECTORY}/{RUN}_{n_clusters}.txt",
                          index_col = 0).loc[filtered_GOset]
    pvalues.columns = map(int, pvalues.columns)

    qvalue_th   = get_qvalue_threshold(pvalues, annotated_cluster_list, gene2GOset)
    enrichments = pvalues < qvalue_th

    cluster_coverages[count] = sum(enrichments.any(axis=0)) / n_clusters
    GO_coverages[     count] = sum(enrichments.any(axis=1)) / len(filtered_GOset)
    gene_coverages[   count] = sum(nb_genes_enriched_in_cluster(enrichment_s,
                                                                annotated_cluster_list[idx],
                                                                gene2GOset)
          for idx, enrichment_s in enrichments.iteritems()) / len(annotated_genes)
    
    break

---

# Testing

In [9]:
set(pd.read_csv(Paths(in_parms).ANNOTATION_FILE).GO_ID) - set(GO2geneset_s.index)

{'GO:0071468',
 'GO:0009166',
 'GO:0071398',
 'GO:0070184',
 'GO:0055092',
 'GO:0090364',
 'GO:0046134',
 'GO:0042148',
 'GO:0061492',
 'GO:0070613',
 'GO:0000493',
 'GO:2001245',
 'GO:0034402',
 'GO:0018208',
 'GO:2001124',
 'GO:0060963',
 'GO:1990546',
 'GO:0043577',
 'GO:0070879',
 'GO:1900409',
 'GO:0007187',
 'GO:0031323',
 'GO:0000256',
 'GO:0010691',
 'GO:0000117',
 'GO:0009098',
 'GO:0006343',
 'GO:0022607',
 'GO:0006481',
 'GO:0051039',
 'GO:0034652',
 'GO:0045596',
 'GO:0043103',
 'GO:0034260',
 'GO:0045069',
 'GO:0000965',
 'GO:0061610',
 'GO:0031991',
 'GO:0030656',
 'GO:0019401',
 'GO:0030320',
 'GO:0046688',
 'GO:0010822',
 'GO:1902274',
 'GO:0006657',
 'GO:0010796',
 'GO:0010155',
 'GO:0000394',
 'GO:1903206',
 'GO:0015861',
 'GO:0061739',
 'GO:1990383',
 'GO:0009188',
 'GO:0120031',
 'GO:0071047',
 'GO:0034337',
 'GO:0110136',
 'GO:2001022',
 'GO:0070146',
 'GO:0080009',
 'GO:0090615',
 'GO:0010453',
 'GO:1990248',
 'GO:0006436',
 'GO:0009074',
 'GO:0070198',
 'GO:00064

In [10]:
pvalues.loc['GO:0071468']

KeyError: 'GO:0071468'

In [6]:
pvalues[0][pvalues[0] < 0.05]

Series([], Name: 0, dtype: float64)

In [9]:
cluster_list = annotated_cluster_list

correction = InputParameters.CORRECTION
alpha      = InputParameters.ALPHA
# union of gene2GOset across all genes in a cluster
def cluster_to_GOset(cluster):
    if len(cluster) == 0:
        return list()
    else:
        return set.union(*map(gene2GOset.get, cluster))

relevant_pvalues = (list(pvalues[idx_cluster][cluster_to_GOset(cluster)])
                         for idx_cluster,cluster in enumerate(cluster_list))

sorted_pvalues = sorted(p for pvalues_s in relevant_pvalues for p in pvalues_s)

m = len(sorted_pvalues) # number of tests

In [21]:
(~pvalues.isna()).sum().sum()

19736

In [11]:
m

19736

In [20]:
'GO:0000001'in GO2geneset_s 

True